In [ ]:
#Part 2: Data Selection
#First, we will wrangle data on neighborhoods from the well-known LA Times list. 
#We use GeoPy to retrieve the coordinates on this data set.
#We also evaluate our startup costs using market data on land prices in these areas from Zillow
#Leveraging Foursquare's location API, we can gather detailed information on the price range, popularity, and trends for ramen shops in our neighborhoods